In [ ]:
# app.py

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary Python modules
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
import os
import logging

from pymongo import MongoClient

####################################
# AnimalShelter Class Definition
####################################
class AnimalShelter:
    """ CRUD operations for Animal collection in MongoDB """

    def __init__(self, username, password, db_name, collection_name, host="localhost", port=27017, authSource='admin'):
        # Create Mongo client connection string with authSource
        uri = f"mongodb://{username}:{password}@{host}:{port}/?authSource={authSource}"

        self.client = MongoClient(uri)
        self.database = self.client[db_name]
        self.collection = self.database[collection_name]

    def read(self, query):
        try:
            return list(self.collection.find(query))
        except Exception as e:
            print(f"Error querying documents: {e}")
            return []


###########################
# Setup Logging
###########################
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler("app_debug.log", mode="w")]
)

logging.info("Starting CS-340 Dashboard setup...")

###########################
# Database Connection
###########################
username = "aacuser"
password = "finalpassword123"
database_name = "AAC"
collection_name = "animals"

try:
    logging.info("Attempting MongoDB connection...")
    # Connect with authSource='admin'
    shelter = AnimalShelter(username, password, database_name, collection_name, authSource='admin')
    logging.info("Connected to MongoDB successfully.")
except Exception as e:
    logging.error(f"Failed to connect to MongoDB: {e}")
    shelter = None

###########################
# Load Data from MongoDB
###########################
if shelter:
    try:
        logging.info("Loading initial data from MongoDB...")
        initial_results = shelter.read({})
        df = pd.DataFrame.from_records(initial_results)
        df.drop(columns=['_id'], inplace=True, errors='ignore')  # Remove '_id' column if present
        logging.info("Initial data loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading initial data: {e}")
        # Create an empty DataFrame if initial load fails
        df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
else:
    # If no connection, create an empty DataFrame
    df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
    logging.warning("Empty DataFrame created due to MongoDB connection failure.")

###########################
# Load Grazioso Salvare Logo
###########################
image_filename = 'logo.png'  # Ensure 'logo.png' is in the current working directory
if os.path.exists(image_filename):
    with open(image_filename, 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode()
    logging.info("Logo image loaded successfully.")
else:
    encoded_image = None
    logging.warning("Logo image not found. The dashboard will load without a logo.")

###########################
# Dashboard Layout
###########################
app = JupyterDash(__name__)
logging.info("Initializing Dashboard layout...")

app.layout = html.Div([
    # Title and Logo
    html.Center(html.B(html.H1('CS-340 Dashboard - Grazioso Salvare'))),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}' if encoded_image else '')),
    html.Hr(),

    # Rescue Type Filter
    html.Label("Filter by Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Show All', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'block'}
    ),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable="single",
        selected_rows=[0],
        style_cell={'textAlign': 'left', 'padding': '5px', 'font_family': 'Arial', 'font_size': '12px'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Hr(),

    # Row for Chart and Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

###########################
# Callbacks
###########################

@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_data(filter_type):
    """Updates the data table based on the selected rescue type filter."""
    if not shelter:
        logging.error("MongoDB connection unavailable in update_data callback.")
        return []

    # Define query based on filter
    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever", "Newfoundland", "Otterhound"]}}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}
    else:
        query = {}

    logging.info(f"Executing MongoDB query: {query}")
    try:
        result = shelter.read(query)
        logging.info(f"Query returned {len(result)} records.")
        df_filtered = pd.DataFrame.from_records(result).drop(columns=['_id'], errors='ignore')
        return df_filtered.to_dict('records')
    except Exception as e:
        logging.error(f"Error executing MongoDB query: {e}")
        return []

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    """Updates the map based on selected rows."""
    if not viewData or len(viewData) == 0:
        logging.warning("No data available for map display.")
        return html.P("No data available.")

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    lat = dff.iloc[row].get('location_lat', 30.75)
    lon = dff.iloc[row].get('location_long', -97.48)
    breed = dff.iloc[row].get('breed', 'Unknown Breed')
    name = dff.iloc[row].get('name', 'Unknown')
    age = dff.iloc[row].get('age_upon_outcome', 'Unknown')

    logging.info(f"Updating map with coordinates: ({lat}, {lon})")
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H4("Animal Details"),
                    html.P(f"Name: {name}"),
                    html.P(f"Age: {age}")
                ])
            ])
        ])
    ]

###########################
# Run Server
###########################
if __name__ == '__main__':
    logging.info("Starting the dashboard server...")
    app.run_server(mode="external", debug=True, port=8050)


2024-12-16 04:48:32,156 [INFO]: Starting CS-340 Dashboard setup...
2024-12-16 04:48:32,157 [INFO]: Attempting MongoDB connection...
2024-12-16 04:48:32,161 [INFO]: Connected to MongoDB successfully.
2024-12-16 04:48:32,162 [INFO]: Loading initial data from MongoDB...


In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
import os
import logging

# Import CRUD Python module
from animal_shelter import AnimalShelter  # Update with your module name

###########################
# Setup Logging
###########################
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

###########################
# Database Connection
###########################
# Replace with your MongoDB credentials
username = "aacuser"  # Ensure this matches your MongoDB user
password = "changeme"  # Replace with correct password

try:
    shelter = AnimalShelter(username, password)
    logging.info("Connected to MongoDB successfully.")
except Exception as e:
    logging.error(f"Error connecting to MongoDB: {e}")

###########################
# Load Initial Data
###########################
try:
    df = pd.DataFrame.from_records(shelter.read({}))  # Load all data
    df.drop(columns=['_id'], inplace=True, errors='ignore')  # Drop '_id' column if it exists
    logging.info("Data loaded successfully.")
except Exception as e:
    logging.error(f"Error loading data: {e}")
    df = pd.DataFrame()  # Fallback to empty DataFrame

###########################
# Load Grazioso Salvare Logo
###########################
image_filename = 'Grazioso_Salvare_Logo.png'  # Ensure the logo is in the same directory
if os.path.exists(image_filename):
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
    logging.info("Logo image loaded successfully.")
else:
    logging.warning("Logo image not found.")
    encoded_image = None

###########################
# Dashboard Layout
###########################
app = JupyterDash(__name__)

app.layout = html.Div([
    # Title and Logo
    html.Center(html.B(html.H1('CS-340 Dashboard - Grazioso Salvare'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()) if encoded_image else '')),
    html.Hr(),
    
    # Rescue Type Filter
    html.Label("Filter by Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Show All', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'block'}
    ),
    html.Hr(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable="single",
        selected_rows=[0],
        style_cell={'textAlign': 'left', 'padding': '5px', 'font_family': 'Arial', 'font_size': '12px'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Hr(),
    
    # Row for Chart and Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

###########################
# Callbacks
###########################

# Callback for Filtering Data
@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_data(filter_type):
    """
    Updates the data table based on the selected filter.
    """
    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever", "Newfoundland", "Otterhound"]}, "age_upon_outcome_in_weeks": {"$lt": 156}}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "age_upon_outcome_in_weeks": {"$gte": 156, "$lte": 300}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
    else:
        query = {}

    result = shelter.read(query)
    return pd.DataFrame.from_records(result).drop(columns=['_id'], errors='ignore').to_dict('records')

# Callback for Updating Map
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    """
    Updates the map based on selected rows.
    """
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H4("Animal Details"),
                    html.P(f"Name: {dff.iloc[row]['name']}"),
                    html.P(f"Age: {dff.iloc[row]['age_upon_outcome']}")
                ])
            ])
        ])
    ]

###########################
# Run Server
###########################
if __name__ == '__main__':
    logging.info("Starting the dashboard...")
    app.run_server(mode="external", debug=True, port=0)  # Open in a new window on an available port


Exception in thread Thread-21:
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/usr/local/anaconda/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda/lib/python3.9/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/usr/local/anaconda/lib/python3.9/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/usr/local/anaconda/lib/python3.9/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/usr/local/anaconda/lib/python3.9/site-packages/six.py", line 719, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.9/site-packages/retrying.py", line 200, in call
    attempt = Attempt(fn(*args, **kwargs), attempt_number, False)
  File "/usr/local/anaconda/lib/python3.9/site-p

In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary Python modules
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
import os
import logging

# Import CRUD Python module
from animal_shelter import AnimalShelter  # Ensure this module has correct credentials

###########################
# Setup Logging
###########################
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler("app_debug.log", mode="w")]
)

logging.info("Starting CS-340 Dashboard setup...")

###########################
# Database Connection
###########################
# Replace these credentials with correct MongoDB user and password
username = "aacuser"
password = "changeme"

try:
    logging.info("Attempting MongoDB connection...")
    shelter = AnimalShelter(username, password)
    logging.info("Connected to MongoDB successfully.")
except Exception as e:
    logging.error(f"Failed to connect to MongoDB: {e}")
    shelter = None

###########################
# Load Data from MongoDB
###########################
if shelter:
    try:
        logging.info("Loading initial data from MongoDB...")
        df = pd.DataFrame.from_records(shelter.read({}))
        df.drop(columns=['_id'], inplace=True, errors='ignore')  # Remove '_id' column
        logging.info("Initial data loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading initial data: {e}")
        df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
else:
    df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
    logging.warning("Empty DataFrame created due to MongoDB connection failure.")

###########################
# Load Grazioso Salvare Logo
###########################
image_filename = 'logo.png'  # Ensure 'logo.png' exists in the current directory
if os.path.exists(image_filename):
    with open(image_filename, 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode()
    logging.info("Logo image loaded successfully.")
else:
    encoded_image = None
    logging.warning("Logo image not found. The dashboard will load without a logo.")

###########################
# Dashboard Layout
###########################
app = JupyterDash(__name__)
logging.info("Initializing Dashboard layout...")

app.layout = html.Div([
    # Title and Logo
    html.Center(html.B(html.H1('CS-340 Dashboard - Grazioso Salvare'))),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}' if encoded_image else '')),
    html.Hr(),

    # Rescue Type Filter
    html.Label("Filter by Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Show All', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'block'}
    ),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable="single",
        selected_rows=[0],
        style_cell={'textAlign': 'left', 'padding': '5px', 'font_family': 'Arial', 'font_size': '12px'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Hr(),

    # Row for Chart and Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

###########################
# Callbacks
###########################

# Callback for Filtering Data
@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_data(filter_type):
    """Updates the data table based on the selected rescue type filter."""
    if not shelter:
        logging.error("MongoDB connection unavailable in update_data callback.")
        return []

    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever", "Newfoundland", "Otterhound"]}}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}
    else:
        query = {}

    logging.info(f"Executing MongoDB query: {query}")
    try:
        result = shelter.read(query)
        logging.info(f"Query returned {len(result)} records.")
        return pd.DataFrame.from_records(result).drop(columns=['_id'], errors='ignore').to_dict('records')
    except Exception as e:
        logging.error(f"Error executing MongoDB query: {e}")
        return []

# Callback for Updating Map
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    """Updates the map based on selected rows."""
    if not viewData or len(viewData) == 0:
        logging.warning("No data available for map display.")
        return html.P("No data available.")

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    logging.info(f"Updating map with coordinates: ({dff.iloc[row]['location_lat']}, {dff.iloc[row]['location_long']})")
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H4("Animal Details"),
                    html.P(f"Name: {dff.iloc[row]['name']}"),
                    html.P(f"Age: {dff.iloc[row]['age_upon_outcome']}")
                ])
            ])
        ])
    ]

###########################
# Run Server
###########################
if __name__ == '__main__':
    logging.info("Starting the dashboard server...")
    app.run_server(mode="external", debug=True, port=8050)


2024-12-16 04:24:05,070 - Starting CS-340 Dashboard setup...
2024-12-16 04:24:05,071 - Attempting MongoDB connection...
2024-12-16 04:24:05,074 - Connected to MongoDB successfully.
2024-12-16 04:24:05,074 - Loading initial data from MongoDB...
2024-12-16 04:24:05,087 - Initial data loaded successfully.
2024-12-16 04:24:05,090 - Logo image loaded successfully.
2024-12-16 04:24:05,097 - Initializing Dashboard layout...
2024-12-16 04:24:05,099 - Starting the dashboard server...


Initializing MongoDB connection...
Connected to MongoDB successfully.
Attempting to read with query: {}
Error querying documents: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}
Dash app running on http://127.0.0.1:8050/


2024-12-16 04:24:16,344 - Executing MongoDB query: {}
2024-12-16 04:24:16,349 - Query returned 0 records.
2024-12-16 04:24:16,360 - No data available for map display.
2024-12-16 04:24:16,427 - No data available for map display.


Attempting to read with query: {}
Error querying documents: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}


2024-12-16 04:24:20,040 - Executing MongoDB query: {'animal_type': 'Dog', 'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}}
2024-12-16 04:24:20,085 - Query returned 0 records.


Attempting to read with query: {'animal_type': 'Dog', 'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}}
Error querying documents: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}


2024-12-16 04:24:20,756 - Executing MongoDB query: {'animal_type': 'Dog', 'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}}
2024-12-16 04:24:20,788 - Query returned 0 records.


Attempting to read with query: {'animal_type': 'Dog', 'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}}
Error querying documents: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}


2024-12-16 04:24:21,499 - Executing MongoDB query: {'animal_type': 'Dog', 'breed': {'$in': ['Labrador Retriever', 'Newfoundland', 'Otterhound']}}
2024-12-16 04:24:21,519 - Query returned 0 records.


Attempting to read with query: {'animal_type': 'Dog', 'breed': {'$in': ['Labrador Retriever', 'Newfoundland', 'Otterhound']}}
Error querying documents: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}


In [5]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary Python modules
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
import os
import logging

# Import CRUD Python module
from animal_shelter import AnimalShelter  # Ensure this module has correct credentials and constructor signature

###########################
# Setup Logging
###########################
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler("app_debug.log", mode="w")]
)

logging.info("Starting CS-340 Dashboard setup...")

###########################
# Database Connection
###########################
# Replace these credentials with correct MongoDB user, password, and ensure the user has access
username = "aacuser"
password = "changeme"
database_name = "AAC"
collection_name = "animals"

try:
    logging.info("Attempting MongoDB connection...")
    # Ensure that your AnimalShelter constructor matches these parameters:
    # AnimalShelter(username, password, database, collection)
    shelter = AnimalShelter(username, password, database_name, collection_name)
    logging.info("Connected to MongoDB successfully.")
except Exception as e:
    logging.error(f"Failed to connect to MongoDB: {e}")
    shelter = None

###########################
# Load Data from MongoDB
###########################
if shelter:
    try:
        logging.info("Loading initial data from MongoDB...")
        initial_results = shelter.read({})
        df = pd.DataFrame.from_records(initial_results)
        df.drop(columns=['_id'], inplace=True, errors='ignore')  # Remove '_id' column if present
        logging.info("Initial data loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading initial data: {e}")
        # Create an empty DataFrame if initial load fails
        df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
else:
    # If no connection, create an empty DataFrame
    df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
    logging.warning("Empty DataFrame created due to MongoDB connection failure.")

###########################
# Load Grazioso Salvare Logo
###########################
image_filename = 'logo.png'  # Ensure 'logo.png' is in the current working directory
if os.path.exists(image_filename):
    with open(image_filename, 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode()
    logging.info("Logo image loaded successfully.")
else:
    encoded_image = None
    logging.warning("Logo image not found. The dashboard will load without a logo.")

###########################
# Dashboard Layout
###########################
app = JupyterDash(__name__)
logging.info("Initializing Dashboard layout...")

app.layout = html.Div([
    # Title and Logo
    html.Center(html.B(html.H1('CS-340 Dashboard - Grazioso Salvare'))),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}' if encoded_image else '')),
    html.Hr(),

    # Rescue Type Filter
    html.Label("Filter by Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Show All', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'block'}
    ),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable="single",
        selected_rows=[0],
        style_cell={'textAlign': 'left', 'padding': '5px', 'font_family': 'Arial', 'font_size': '12px'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Hr(),

    # Row for Chart and Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

###########################
# Callbacks
###########################

# Callback for Filtering Data
@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_data(filter_type):
    """Updates the data table based on the selected rescue type filter."""
    if not shelter:
        logging.error("MongoDB connection unavailable in update_data callback.")
        return []

    # Define query based on filter
    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever", "Newfoundland", "Otterhound"]}}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}
    else:
        query = {}

    logging.info(f"Executing MongoDB query: {query}")
    try:
        result = shelter.read(query)
        logging.info(f"Query returned {len(result)} records.")
        df_filtered = pd.DataFrame.from_records(result).drop(columns=['_id'], errors='ignore')
        return df_filtered.to_dict('records')
    except Exception as e:
        logging.error(f"Error executing MongoDB query: {e}")
        return []

# Callback for Updating Map
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    """Updates the map based on selected rows."""
    if not viewData or len(viewData) == 0:
        logging.warning("No data available for map display.")
        return html.P("No data available.")

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    lat = dff.iloc[row].get('location_lat', 30.75)
    lon = dff.iloc[row].get('location_long', -97.48)
    breed = dff.iloc[row].get('breed', 'Unknown Breed')
    name = dff.iloc[row].get('name', 'Unknown')
    age = dff.iloc[row].get('age_upon_outcome', 'Unknown')

    logging.info(f"Updating map with coordinates: ({lat}, {lon})")
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H4("Animal Details"),
                    html.P(f"Name: {name}"),
                    html.P(f"Age: {age}")
                ])
            ])
        ])
    ]

###########################
# Run Server
###########################
if __name__ == '__main__':
    logging.info("Starting the dashboard server...")
    # For Jupyter environments, you might prefer mode='inline' or mode='jupyterlab'
    app.run_server(mode="external", debug=True, port=8050)


2024-12-16 04:28:19,620 - Starting CS-340 Dashboard setup...
2024-12-16 04:28:19,621 - Attempting MongoDB connection...
2024-12-16 04:28:19,621 - Failed to connect to MongoDB: __init__() takes 3 positional arguments but 5 were given
2024-12-16 04:28:19,624 - Empty DataFrame created due to MongoDB connection failure.
2024-12-16 04:28:19,626 - Logo image loaded successfully.
2024-12-16 04:28:19,634 - Initializing Dashboard layout...
2024-12-16 04:28:19,637 - Starting the dashboard server...


Dash app running on http://127.0.0.1:8050/


2024-12-16 04:28:26,271 - MongoDB connection unavailable in update_data callback.
2024-12-16 04:28:26,276 - No data available for map display.
2024-12-16 04:28:26,375 - No data available for map display.
2024-12-16 04:28:29,873 - MongoDB connection unavailable in update_data callback.
2024-12-16 04:28:30,649 - MongoDB connection unavailable in update_data callback.


In [ ]:
# app.py

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary Python modules
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
import os
import logging

from pymongo import MongoClient

####################################
# AnimalShelter Class Definition
####################################
class AnimalShelter:
    """ CRUD operations for Animal collection in MongoDB """

    def __init__(self, username, password, db_name, collection_name, host="localhost", port=27017, authSource='admin'):
        # Create Mongo client connection string with authSource
        uri = f"mongodb://{username}:{password}@{host}:{port}/?authSource={authSource}"

        self.client = MongoClient(uri)
        self.database = self.client[db_name]
        self.collection = self.database[collection_name]

    def read(self, query):
        try:
            return list(self.collection.find(query))
        except Exception as e:
            print(f"Error querying documents: {e}")
            return []


###########################
# Setup Logging
###########################
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler("app_debug.log", mode="w")]
)

logging.info("Starting CS-340 Dashboard setup...")

###########################
# Database Connection
###########################
username = "aacuser"
password = "finalpassword123"
database_name = "AAC"
collection_name = "animals"

try:
    logging.info("Attempting MongoDB connection...")
    # Connect with authSource='admin'
    shelter = AnimalShelter(username, password, database_name, collection_name, authSource='admin')
    logging.info("Connected to MongoDB successfully.")
except Exception as e:
    logging.error(f"Failed to connect to MongoDB: {e}")
    shelter = None

###########################
# Load Data from MongoDB
###########################
if shelter:
    try:
        logging.info("Loading initial data from MongoDB...")
        initial_results = shelter.read({})
        df = pd.DataFrame.from_records(initial_results)
        df.drop(columns=['_id'], inplace=True, errors='ignore')  # Remove '_id' column if present
        logging.info("Initial data loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading initial data: {e}")
        # Create an empty DataFrame if initial load fails
        df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
else:
    # If no connection, create an empty DataFrame
    df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
    logging.warning("Empty DataFrame created due to MongoDB connection failure.")

###########################
# Load Grazioso Salvare Logo
###########################
image_filename = 'logo.png'  # Ensure 'logo.png' is in the current working directory
if os.path.exists(image_filename):
    with open(image_filename, 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode()
    logging.info("Logo image loaded successfully.")
else:
    encoded_image = None
    logging.warning("Logo image not found. The dashboard will load without a logo.")

###########################
# Dashboard Layout
###########################
app = JupyterDash(__name__)
logging.info("Initializing Dashboard layout...")

app.layout = html.Div([
    # Title and Logo
    html.Center(html.B(html.H1('CS-340 Dashboard - Grazioso Salvare'))),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}' if encoded_image else '')),
    html.Hr(),

    # Rescue Type Filter
    html.Label("Filter by Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Show All', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'block'}
    ),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable="single",
        selected_rows=[0],
        style_cell={'textAlign': 'left', 'padding': '5px', 'font_family': 'Arial', 'font_size': '12px'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Hr(),

    # Row for Chart and Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

###########################
# Callbacks
###########################

@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_data(filter_type):
    """Updates the data table based on the selected rescue type filter."""
    if not shelter:
        logging.error("MongoDB connection unavailable in update_data callback.")
        return []

    # Define query based on filter
    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever", "Newfoundland", "Otterhound"]}}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}
    else:
        query = {}

    logging.info(f"Executing MongoDB query: {query}")
    try:
        result = shelter.read(query)
        logging.info(f"Query returned {len(result)} records.")
        df_filtered = pd.DataFrame.from_records(result).drop(columns=['_id'], errors='ignore')
        return df_filtered.to_dict('records')
    except Exception as e:
        logging.error(f"Error executing MongoDB query: {e}")
        return []

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    """Updates the map based on selected rows."""
    if not viewData or len(viewData) == 0:
        logging.warning("No data available for map display.")
        return html.P("No data available.")

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    lat = dff.iloc[row].get('location_lat', 30.75)
    lon = dff.iloc[row].get('location_long', -97.48)
    breed = dff.iloc[row].get('breed', 'Unknown Breed')
    name = dff.iloc[row].get('name', 'Unknown')
    age = dff.iloc[row].get('age_upon_outcome', 'Unknown')

    logging.info(f"Updating map with coordinates: ({lat}, {lon})")
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H4("Animal Details"),
                    html.P(f"Name: {name}"),
                    html.P(f"Age: {age}")
                ])
            ])
        ])
    ]

###########################
# Run Server
###########################
if __name__ == '__main__':
    logging.info("Starting the dashboard server...")
    app.run_server(mode="external", debug=True, port=8050)


In [3]:
# app.py

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary Python modules
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
import os
import logging

from pymongo import MongoClient

####################################
# AnimalShelter Class Definition
####################################
class AnimalShelter:
    """ CRUD operations for Animal collection in MongoDB """

    def __init__(self, username, password, db_name, collection_name, host="localhost", port=27017, authSource='admin'):
        # Create Mongo client connection string with authSource
        uri = f"mongodb://{username}:{password}@{host}:{port}/?authSource={authSource}"

        self.client = MongoClient(uri)
        self.database = self.client[db_name]
        self.collection = self.database[collection_name]

    def read(self, query):
        try:
            return list(self.collection.find(query))
        except Exception as e:
            print(f"Error querying documents: {e}")
            return []


###########################
# Setup Logging
###########################
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler("app_debug.log", mode="w")]
)

logging.info("Starting CS-340 Dashboard setup...")

###########################
# Database Connection
###########################
username = "aacuser"
password = "finalpassword123"
database_name = "AAC"
collection_name = "animals"

try:
    logging.info("Attempting MongoDB connection...")
    # Connect with authSource='admin'
    shelter = AnimalShelter(username, password, database_name, collection_name, authSource='admin')
    logging.info("Connected to MongoDB successfully.")
except Exception as e:
    logging.error(f"Failed to connect to MongoDB: {e}")
    shelter = None

###########################
# Load Data from MongoDB
###########################
if shelter:
    try:
        logging.info("Loading initial data from MongoDB...")
        initial_results = shelter.read({})
        df = pd.DataFrame.from_records(initial_results)
        df.drop(columns=['_id'], inplace=True, errors='ignore')  # Remove '_id' column if present
        logging.info("Initial data loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading initial data: {e}")
        # Create an empty DataFrame if initial load fails
        df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
else:
    # If no connection, create an empty DataFrame
    df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
    logging.warning("Empty DataFrame created due to MongoDB connection failure.")

###########################
# Load Grazioso Salvare Logo
###########################
image_filename = 'logo.png'  # Ensure 'logo.png' is in the current working directory
if os.path.exists(image_filename):
    with open(image_filename, 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode()
    logging.info("Logo image loaded successfully.")
else:
    encoded_image = None
    logging.warning("Logo image not found. The dashboard will load without a logo.")

###########################
# Dashboard Layout
###########################
app = JupyterDash(__name__)
logging.info("Initializing Dashboard layout...")

app.layout = html.Div([
    # Title and Logo
    html.Center(html.B(html.H1('CS-340 Dashboard - Grazioso Salvare'))),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}' if encoded_image else '')),
    html.Hr(),

    # Rescue Type Filter
    html.Label("Filter by Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Show All', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'block'}
    ),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable="single",
        selected_rows=[0],
        style_cell={'textAlign': 'left', 'padding': '5px', 'font_family': 'Arial', 'font_size': '12px'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Hr(),

    # Row for Chart and Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

###########################
# Callbacks
###########################

@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_data(filter_type):
    """Updates the data table based on the selected rescue type filter."""
    if not shelter:
        logging.error("MongoDB connection unavailable in update_data callback.")
        return []

    # Define query based on filter
    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever", "Newfoundland", "Otterhound"]}}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}
    else:
        query = {}

    logging.info(f"Executing MongoDB query: {query}")
    try:
        result = shelter.read(query)
        logging.info(f"Query returned {len(result)} records.")
        df_filtered = pd.DataFrame.from_records(result).drop(columns=['_id'], errors='ignore')
        return df_filtered.to_dict('records')
    except Exception as e:
        logging.error(f"Error executing MongoDB query: {e}")
        return []

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    """Updates the map based on selected rows."""
    if not viewData or len(viewData) == 0:
        logging.warning("No data available for map display.")
        return html.P("No data available.")

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    lat = dff.iloc[row].get('location_lat', 30.75)
    lon = dff.iloc[row].get('location_long', -97.48)
    breed = dff.iloc[row].get('breed', 'Unknown Breed')
    name = dff.iloc[row].get('name', 'Unknown')
    age = dff.iloc[row].get('age_upon_outcome', 'Unknown')

    logging.info(f"Updating map with coordinates: ({lat}, {lon})")
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H4("Animal Details"),
                    html.P(f"Name: {name}"),
                    html.P(f"Age: {age}")
                ])
            ])
        ])
    ]

###########################
# Run Server
###########################
if __name__ == '__main__':
    logging.info("Starting the dashboard server...")
    app.run_server(mode="external", debug=True, port=8050)


2024-12-16 04:49:34,122 [INFO]: Starting CS-340 Dashboard setup...
2024-12-16 04:49:34,123 [INFO]: Attempting MongoDB connection...
2024-12-16 04:49:34,124 [INFO]: Connected to MongoDB successfully.
2024-12-16 04:49:34,125 [INFO]: Loading initial data from MongoDB...
2024-12-16 04:50:04,182 [INFO]: Initial data loaded successfully.
2024-12-16 04:50:04,184 [INFO]: Logo image loaded successfully.
2024-12-16 04:50:04,189 [INFO]: Initializing Dashboard layout...
2024-12-16 04:50:04,191 [INFO]: Starting the dashboard server...


Error querying documents: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 675fb15eba355a7b34d9df4f, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>
Dash app running on http://127.0.0.1:8050/


2024-12-16 04:50:09,071 [INFO]: Executing MongoDB query: {}
2024-12-16 04:50:09,082 [WARNING]: No data available for map display.
2024-12-16 04:50:09,160 [WARNING]: No data available for map display.
2024-12-16 04:50:11,943 [INFO]: Executing MongoDB query: {'animal_type': 'Dog', 'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}}
2024-12-16 04:50:12,699 [INFO]: Executing MongoDB query: {'animal_type': 'Dog', 'breed': {'$in': ['Labrador Retriever', 'Newfoundland', 'Otterhound']}}
2024-12-16 04:50:13,296 [INFO]: Executing MongoDB query: {'animal_type': 'Dog', 'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}}
2024-12-16 04:50:39,250 [INFO]: Query returned 0 records.


Error querying documents: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 675fb15eba355a7b34d9df4f, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>


2024-12-16 04:50:42,257 [INFO]: Query returned 0 records.


Error querying documents: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 675fb15eba355a7b34d9df4f, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>


2024-12-16 04:50:42,760 [INFO]: Query returned 0 records.


Error querying documents: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 675fb15eba355a7b34d9df4f, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>


2024-12-16 04:50:43,764 [INFO]: Query returned 0 records.


Error querying documents: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 675fb15eba355a7b34d9df4f, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>


In [1]:
# app.py

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary Python modules
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import base64
import os
import logging

from pymongo import MongoClient

####################################
# AnimalShelter Class Definition
####################################
class AnimalShelter:
    """ CRUD operations for Animal collection in MongoDB """

    def __init__(self, username, password, db_name, collection_name, host="nv-desktop-services.apporto.com", port=32941, authSource='admin'):
        # Construct the MongoDB connection URI
        uri = f"mongodb://{username}:{password}@{host}:{port}/?authSource={authSource}"
        self.client = MongoClient(uri, serverSelectionTimeoutMS=30000)  # 30s timeout
        self.database = self.client[db_name]
        self.collection = self.database[collection_name]

    def read(self, query):
        try:
            return list(self.collection.find(query))
        except Exception as e:
            print(f"Error querying documents: {e}")
            return []


###########################
# Setup Logging
###########################
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler("app_debug.log", mode="w")]
)

logging.info("Starting CS-340 Dashboard setup...")

###########################
# Database Connection
###########################
username = "aacuser"
password = "finalpassword123"
database_name = "AAC"
collection_name = "animals"

try:
    logging.info("Attempting MongoDB connection...")
    shelter = AnimalShelter(username, password, database_name, collection_name, host="nv-desktop-services.apporto.com", port=32941, authSource='admin')
    # Test the connection by attempting to retrieve server info
    shelter.client.server_info()
    logging.info("Connected to MongoDB successfully.")
except Exception as e:
    logging.error(f"Failed to connect to MongoDB: {e}")
    shelter = None

###########################
# Load Data from MongoDB
###########################
if shelter:
    try:
        logging.info("Loading initial data from MongoDB...")
        initial_results = shelter.read({})
        df = pd.DataFrame.from_records(initial_results)
        df.drop(columns=['_id'], inplace=True, errors='ignore')  # Remove '_id' column if present
        logging.info("Initial data loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading initial data: {e}")
        df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
else:
    # If no connection, create an empty DataFrame
    df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
    logging.warning("Empty DataFrame created due to MongoDB connection failure.")

###########################
# Load Grazioso Salvare Logo
###########################
image_filename = 'logo.png'  # Ensure 'logo.png' is in the current directory if you have it
if os.path.exists(image_filename):
    with open(image_filename, 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode()
    logging.info("Logo image loaded successfully.")
else:
    encoded_image = None
    logging.warning("Logo image not found. The dashboard will load without a logo.")

###########################
# Dashboard Layout
###########################
app = JupyterDash(__name__)
logging.info("Initializing Dashboard layout...")

app.layout = html.Div([
    # Title and Logo
    html.Center(html.B(html.H1('CS-340 Dashboard - Grazioso Salvare'))),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}' if encoded_image else '')),
    html.Hr(),

    # Rescue Type Filter
    html.Label("Filter by Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Show All', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'block'}
    ),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable="single",
        selected_rows=[0],
        style_cell={'textAlign': 'left', 'padding': '5px', 'font_family': 'Arial', 'font_size': '12px'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Hr(),

    # Row for Chart and Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

###########################
# Callbacks
###########################

@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_data(filter_type):
    """Updates the data table based on the selected rescue type filter."""
    if not shelter:
        logging.error("MongoDB connection unavailable in update_data callback.")
        return []

    # Define query based on filter
    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever", "Newfoundland", "Otterhound"]}}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}
    else:
        query = {}

    logging.info(f"Executing MongoDB query: {query}")
    try:
        result = shelter.read(query)
        logging.info(f"Query returned {len(result)} records.")
        df_filtered = pd.DataFrame.from_records(result).drop(columns=['_id'], errors='ignore')
        return df_filtered.to_dict('records')
    except Exception as e:
        logging.error(f"Error executing MongoDB query: {e}")
        return []

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    """Updates the map based on selected rows."""
    if not viewData or len(viewData) == 0:
        logging.warning("No data available for map display.")
        return html.P("No data available.")

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    lat = dff.iloc[row].get('location_lat', 30.75)
    lon = dff.iloc[row].get('location_long', -97.48)
    breed = dff.iloc[row].get('breed', 'Unknown Breed')
    name = dff.iloc[row].get('name', 'Unknown')
    age = dff.iloc[row].get('age_upon_outcome', 'Unknown')

    logging.info(f"Updating map with coordinates: ({lat}, {lon})")
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H4("Animal Details"),
                    html.P(f"Name: {name}"),
                    html.P(f"Age: {age}")
                ])
            ])
        ])
    ]

###########################
# Run Server
###########################
if __name__ == '__main__':
    logging.info("Starting the dashboard server...")
    # Running externally. Adjust mode if needed (e.g. mode="inline" if in Jupyter)
    app.run_server(mode="external", debug=True, port=8050)


2024-12-16 05:00:59,370 [INFO]: Starting CS-340 Dashboard setup...
2024-12-16 05:00:59,372 [INFO]: Attempting MongoDB connection...
2024-12-16 05:00:59,409 [INFO]: Connected to MongoDB successfully.
2024-12-16 05:00:59,410 [INFO]: Loading initial data from MongoDB...
2024-12-16 05:00:59,518 [INFO]: Initial data loaded successfully.
2024-12-16 05:00:59,524 [INFO]: Logo image loaded successfully.
2024-12-16 05:00:59,531 [INFO]: Initializing Dashboard layout...
2024-12-16 05:00:59,592 [INFO]: Starting the dashboard server...


Dash app running on http://127.0.0.1:8050/


2024-12-16 05:01:06,297 [INFO]: Executing MongoDB query: {}
2024-12-16 05:01:06,445 [INFO]: Query returned 10003 records.
2024-12-16 05:01:06,603 [WARNING]: No data available for map display.
2024-12-16 05:01:08,270 [INFO]: Updating map with coordinates: (30.6525984560228, -97.7419963476444)
2024-12-16 05:02:00,611 [INFO]: Executing MongoDB query: {'animal_type': 'Dog', 'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}}
2024-12-16 05:02:00,629 [INFO]: Query returned 39 records.
2024-12-16 05:02:01,052 [INFO]: Updating map with coordinates: (30.4861149595305, -97.2691161890552)
2024-12-16 05:02:02,276 [INFO]: Executing MongoDB query: {'animal_type': 'Dog', 'breed': {'$in': ['Labrador Retriever', 'Newfoundland', 'Otterhound']}}
2024-12-16 05:02:02,297 [INFO]: Query returned 18 records.
2024-12-16 05:02:02,366 [INFO]: Updating map with coordinates: (30.3879952345421, -97.4272937100246)
2024-12-16 05:02:03,457 [INFO]: Executing Mongo

In [2]:
# app.py

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary Python modules
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import pandas as pd
import base64
import os
import logging

from pymongo import MongoClient

####################################
# AnimalShelter Class Definition
####################################
class AnimalShelter:
    """ CRUD operations for Animal collection in MongoDB """

    def __init__(self, username, password, db_name, collection_name, host="nv-desktop-services.apporto.com", port=32941, authSource='admin'):
        uri = f"mongodb://{username}:{password}@{host}:{port}/?authSource={authSource}"
        self.client = MongoClient(uri, serverSelectionTimeoutMS=30000)
        self.database = self.client[db_name]
        self.collection = self.database[collection_name]

    def read(self, query):
        try:
            return list(self.collection.find(query))
        except Exception as e:
            logging.error(f"Error querying documents: {e}")
            return []

###########################
# Setup Logging
###########################
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler("app_debug.log", mode="w")]
)

logging.info("Starting CS-340 Dashboard setup...")

###########################
# Database Connection
###########################
username = "aacuser"
password = "finalpassword123"
database_name = "AAC"
collection_name = "animals"

try:
    logging.info("Attempting MongoDB connection...")
    shelter = AnimalShelter(username, password, database_name, collection_name)
    shelter.client.server_info()  # Test the connection
    logging.info("Connected to MongoDB successfully.")
except Exception as e:
    logging.error(f"Failed to connect to MongoDB: {e}")
    shelter = None

###########################
# Load Data from MongoDB
###########################
if shelter:
    try:
        logging.info("Loading initial data from MongoDB...")
        initial_results = shelter.read({})
        df = pd.DataFrame.from_records(initial_results)
        df.drop(columns=['_id'], inplace=True, errors='ignore')
        logging.info("Initial data loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading initial data: {e}")
        df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
else:
    df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
    logging.warning("Empty DataFrame created due to MongoDB connection failure.")

###########################
# Load Grazioso Salvare Logo
###########################
image_filename = 'logo.png'
if os.path.exists(image_filename):
    with open(image_filename, 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode()
    logging.info("Logo image loaded successfully.")
else:
    encoded_image = None
    logging.warning("Logo image not found.")

###########################
# Dashboard Layout
###########################
app = JupyterDash(__name__)
logging.info("Initializing Dashboard layout...")

app.layout = html.Div([
    # Title and Logo
    html.Center(html.B(html.H1('CS-340 Dashboard - Grazioso Salvare'))),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}' if encoded_image else '')),
    html.Hr(),

    # Rescue Type Filter and Page Size Input
    html.Div([
        html.Label("Filter by Rescue Type:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Show All', 'value': 'all'}
            ],
            value='all',
            labelStyle={'display': 'block'}
        ),
        html.Label("Page Size:"),
        dcc.Input(id='page-size', type='number', value=10, min=5, max=50, step=5, debounce=True),
    ]),
    html.Hr(),

    # Data Table with Spinner
    dcc.Loading(
        id="loading-table",
        type="circle",
        children=[
            dash_table.DataTable(
                id='datatable-id',
                columns=[{"name": i, "id": i} for i in df.columns],
                data=df.to_dict('records'),
                page_size=10,
                style_table={'overflowX': 'auto'},
                row_selectable="single",
                selected_rows=[0],
                style_cell={'textAlign': 'left', 'padding': '5px', 'font_family': 'Arial', 'font_size': '12px'},
                style_header={'backgroundColor': 'white', 'fontWeight': 'bold'}
            )
        ]
    ),
    html.Br(),
    html.Hr(),

    # Chart and Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

###########################
# Callbacks
###########################
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value'), Input('page-size', 'value')]
)
def update_data(filter_type, page_size):
    if not shelter:
        logging.error("MongoDB connection unavailable.")
        return []

    query = {}
    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever", "Newfoundland", "Otterhound"]}}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "Golden Retriever", "Bloodhound", "Rottweiler"]}}

    logging.info(f"Executing query: {query}")
    results = shelter.read(query)
    logging.info(f"Retrieved {len(results)} records.")
    df_filtered = pd.DataFrame.from_records(results).drop(columns=['_id'], errors='ignore')
    return df_filtered.to_dict('records')

@app.callback(
    Output('map-id', 'children'),
    Input('datatable-id', 'derived_virtual_data'),
    Input('datatable-id', 'derived_virtual_selected_rows')
)
def update_map(viewData, index):
    if not viewData:
        return html.P("No data available.")

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    lat = dff.iloc[row].get('location_lat', 30.75)
    lon = dff.iloc[row].get('location_long', -97.48)
    return dl.Map(center=[lat, lon], zoom=10, style={'height': '500px'}, children=[
        dl.TileLayer(),
        dl.Marker(position=[lat, lon], children=[dl.Tooltip("Selected Location")])
    ])

###########################
# Run Server
###########################
if __name__ == '__main__':
    logging.info("Starting server...")
    app.run_server(mode="external", debug=True, port=8050)


2024-12-16 05:05:39,014 [INFO]: Starting CS-340 Dashboard setup...
2024-12-16 05:05:39,015 [INFO]: Attempting MongoDB connection...
2024-12-16 05:05:39,031 [INFO]: Connected to MongoDB successfully.
2024-12-16 05:05:39,032 [INFO]: Loading initial data from MongoDB...
2024-12-16 05:05:39,183 [INFO]: Initial data loaded successfully.
2024-12-16 05:05:39,187 [INFO]: Logo image loaded successfully.
2024-12-16 05:05:39,196 [INFO]: Initializing Dashboard layout...
2024-12-16 05:05:39,262 [INFO]: Starting server...


Dash app running on http://127.0.0.1:8050/


2024-12-16 05:05:43,652 [INFO]: Executing query: {}
2024-12-16 05:05:43,705 [INFO]: Retrieved 10003 records.
2024-12-16 05:05:53,197 [INFO]: Executing query: {'animal_type': 'Dog', 'breed': {'$in': ['Doberman Pinscher', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}}
2024-12-16 05:05:53,213 [INFO]: Retrieved 13 records.
2024-12-16 05:05:58,541 [INFO]: Executing query: {}
2024-12-16 05:05:58,622 [INFO]: Retrieved 10003 records.
2024-12-16 05:06:00,426 [INFO]: Executing query: {'animal_type': 'Dog', 'breed': {'$in': ['Labrador Retriever', 'Newfoundland', 'Otterhound']}}
2024-12-16 05:06:00,439 [INFO]: Retrieved 18 records.
2024-12-16 05:06:01,858 [INFO]: Executing query: {'animal_type': 'Dog', 'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Siberian Husky', 'Rottweiler']}}
2024-12-16 05:06:01,866 [INFO]: Retrieved 46 records.
2024-12-16 05:06:15,911 [INFO]: Executing query: {}
2024-12-16 05:06:15,967 [INFO]: Retrieved 10003 records.


In [3]:
# app.py

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary Python modules
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import base64
import os
import logging

from pymongo import MongoClient

####################################
# AnimalShelter Class Definition
####################################
class AnimalShelter:
    """ CRUD operations for Animal collection in MongoDB """

    def __init__(self, username, password, db_name, collection_name, host="nv-desktop-services.apporto.com", port=32941, authSource='admin'):
        # Construct the MongoDB connection URI
        uri = f"mongodb://{username}:{password}@{host}:{port}/?authSource={authSource}"
        self.client = MongoClient(uri, serverSelectionTimeoutMS=30000)  # 30s timeout
        self.database = self.client[db_name]
        self.collection = self.database[collection_name]

    def read(self, query):
        """ Fetch documents from MongoDB """
        try:
            return list(self.collection.find(query))
        except Exception as e:
            logging.error(f"Error querying documents: {e}")
            return []


###########################
# Setup Logging
###########################
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler("app_debug.log", mode="w")]
)

logging.info("Starting CS-340 Dashboard setup...")

###########################
# Database Connection
###########################
username = "aacuser"
password = "finalpassword123"
database_name = "AAC"
collection_name = "animals"

try:
    logging.info("Attempting MongoDB connection...")
    shelter = AnimalShelter(username, password, database_name, collection_name, host="nv-desktop-services.apporto.com", port=32941, authSource='admin')
    shelter.client.server_info()  # Test the connection
    logging.info("Connected to MongoDB successfully.")
except Exception as e:
    logging.error(f"Failed to connect to MongoDB: {e}")
    shelter = None

###########################
# Load Data from MongoDB
###########################
if shelter:
    try:
        logging.info("Loading initial data from MongoDB...")
        initial_results = shelter.read({})
        df = pd.DataFrame.from_records(initial_results)
        df.drop(columns=['_id'], inplace=True, errors='ignore')  # Remove '_id' column if present
        logging.info("Initial data loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading initial data: {e}")
        df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
else:
    df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
    logging.warning("Empty DataFrame created due to MongoDB connection failure.")

###########################
# Load Grazioso Salvare Logo
###########################
image_filename = 'logo.png'  # Ensure 'logo.png' is in the current directory
if os.path.exists(image_filename):
    with open(image_filename, 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode()
    logging.info("Logo image loaded successfully.")
else:
    encoded_image = None
    logging.warning("Logo image not found. The dashboard will load without a logo.")

###########################
# Dashboard Layout
###########################
app = JupyterDash(__name__)
logging.info("Initializing Dashboard layout...")

app.layout = html.Div([
    # Title and Logo
    html.Center(html.B(html.H1('CS-340 Dashboard - Grazioso Salvare'))),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}' if encoded_image else '')),
    html.Hr(),

    # Rescue Type Filter
    html.Label("Filter by Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Show All', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'block'}
    ),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable="single",
        selected_rows=[0],
        style_cell={'textAlign': 'left', 'padding': '5px', 'font_family': 'Arial', 'font_size': '12px'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Hr(),

    # Row for Chart and Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

###########################
# Callbacks
###########################

@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_data(filter_type):
    """ Updates the data table based on the selected rescue type filter. """
    if not shelter:
        logging.error("MongoDB connection unavailable in update_data callback.")
        return []

    # Define query based on filter
    query = {}
    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever", "Newfoundland", "Otterhound"]}}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}

    logging.info(f"Executing MongoDB query: {query}")
    try:
        result = shelter.read(query)
        logging.info(f"Query returned {len(result)} records.")
        return pd.DataFrame.from_records(result).drop(columns=['_id'], errors='ignore').to_dict('records')
    except Exception as e:
        logging.error(f"Error executing MongoDB query: {e}")
        return []

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    """ Updates the map based on selected rows. """
    if not viewData or len(viewData) == 0:
        logging.warning("No data available for map display.")
        return html.P("No data available.")

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    try:
        lat = float(dff.iloc[row]['location_lat'])
        lon = float(dff.iloc[row]['location_long'])
    except (KeyError, ValueError, TypeError) as e:
        logging.error(f"Invalid coordinates for map: {e}")
        lat, lon = 30.75, -97.48  # Default coordinates

    logging.info(f"Displaying map for coordinates: ({lat}, {lon})")
    return dl.Map(center=[lat, lon], zoom=10, style={'height': '500px'}, children=[
        dl.TileLayer(),
        dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(f"Coordinates: ({lat}, {lon})")
        ])
    ])

###########################
# Run Server
###########################
if __name__ == '__main__':
    logging.info("Starting the dashboard server...")
    app.run_server(mode="external", debug=True, port=8050)


2024-12-16 05:10:59,632 [INFO]: Starting CS-340 Dashboard setup...
2024-12-16 05:10:59,633 [INFO]: Attempting MongoDB connection...
2024-12-16 05:10:59,649 [INFO]: Connected to MongoDB successfully.
2024-12-16 05:10:59,650 [INFO]: Loading initial data from MongoDB...
2024-12-16 05:10:59,742 [INFO]: Initial data loaded successfully.
2024-12-16 05:10:59,747 [INFO]: Logo image loaded successfully.
2024-12-16 05:10:59,755 [INFO]: Initializing Dashboard layout...
2024-12-16 05:10:59,819 [INFO]: Starting the dashboard server...


Dash app running on http://127.0.0.1:8050/


2024-12-16 05:11:04,510 [INFO]: Executing MongoDB query: {}
2024-12-16 05:11:04,638 [INFO]: Query returned 10003 records.
2024-12-16 05:11:04,839 [WARNING]: No data available for map display.
2024-12-16 05:11:06,059 [INFO]: Displaying map for coordinates: (30.6525984560228, -97.7419963476444)
2024-12-16 05:11:13,554 [INFO]: Executing MongoDB query: {'animal_type': 'Dog', 'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}}
2024-12-16 05:11:13,562 [INFO]: Query returned 39 records.
2024-12-16 05:11:14,009 [INFO]: Displaying map for coordinates: (30.4861149595305, -97.2691161890552)
2024-12-16 05:11:14,741 [INFO]: Executing MongoDB query: {'animal_type': 'Dog', 'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}}
2024-12-16 05:11:14,749 [INFO]: Query returned 47 records.
2024-12-16 05:11:14,830 [INFO]: Displaying map for coordinates: (30.4499994838973, -97.6185491903329)
2

In [4]:
# app.py

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary Python modules
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import base64
import os
import logging
from pymongo import MongoClient

####################################
# AnimalShelter Class Definition
####################################
class AnimalShelter:
    """ CRUD operations for Animal collection in MongoDB """

    def __init__(self, username, password, db_name, collection_name, host="nv-desktop-services.apporto.com", port=32941, authSource='admin'):
        uri = f"mongodb://{username}:{password}@{host}:{port}/?authSource={authSource}"
        self.client = MongoClient(uri, serverSelectionTimeoutMS=30000)
        self.database = self.client[db_name]
        self.collection = self.database[collection_name]

    def read(self, query):
        try:
            return list(self.collection.find(query))
        except Exception as e:
            print(f"Error querying documents: {e}")
            return []


###########################
# Setup Logging
###########################
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler("app_debug.log", mode="w")]
)
logging.info("Starting CS-340 Dashboard setup...")

###########################
# Database Connection
###########################
username = "aacuser"
password = "finalpassword123"
database_name = "AAC"
collection_name = "animals"

try:
    logging.info("Attempting MongoDB connection...")
    shelter = AnimalShelter(username, password, database_name, collection_name)
    shelter.client.server_info()
    logging.info("Connected to MongoDB successfully.")
except Exception as e:
    logging.error(f"Failed to connect to MongoDB: {e}")
    shelter = None

###########################
# Load Data from MongoDB
###########################
if shelter:
    try:
        logging.info("Loading initial data from MongoDB...")
        initial_results = shelter.read({})
        df = pd.DataFrame.from_records(initial_results)
        df.drop(columns=['_id'], inplace=True, errors='ignore')  # Remove '_id' column if present
        logging.info("Initial data loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading initial data: {e}")
        df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
else:
    df = pd.DataFrame(columns=["animal_type", "breed", "name", "age_upon_outcome", "location_lat", "location_long"])
    logging.warning("Empty DataFrame created due to MongoDB connection failure.")

###########################
# Load Grazioso Salvare Logo
###########################
image_filename = 'logo.png'
if os.path.exists(image_filename):
    with open(image_filename, 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode()
    logging.info("Logo image loaded successfully.")
else:
    encoded_image = None
    logging.warning("Logo image not found.")

###########################
# Dashboard Layout
###########################
app = JupyterDash(__name__)
logging.info("Initializing Dashboard layout...")

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard - Grazioso Salvare'))),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}' if encoded_image else '')),
    html.Hr(),
    html.Label("Filter by Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Show All', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'block'}
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable="single",
        selected_rows=[0],
        style_cell={'textAlign': 'left', 'padding': '5px'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Div(id='map-id', style={'height': '500px'})
])

###########################
# Callbacks
###########################

@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_data(filter_type):
    if not shelter:
        logging.error("MongoDB connection unavailable in update_data callback.")
        return []

    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever", "Newfoundland", "Otterhound"]}}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}
    else:
        query = {}

    result = shelter.read(query)
    df_filtered = pd.DataFrame.from_records(result).drop(columns=['_id'], errors='ignore')
    return df_filtered.to_dict('records')

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    if not viewData or len(viewData) == 0:
        return html.P("No data available for map display.")

    dff = pd.DataFrame(viewData)
    row = index[0] if index else 0

    try:
        lat = float(dff.iloc[row]['location_lat'])
        lon = float(dff.iloc[row]['location_long'])
    except (KeyError, ValueError, IndexError):
        lat, lon = 30.75, -97.48  # Default fallback coordinates
        logging.warning("Invalid or missing coordinates; using default.")

    return dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
        dl.TileLayer(),
        dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(dff.iloc[row].get('breed', 'Unknown')),
            dl.Popup([
                html.H4("Animal Details"),
                html.P(f"Name: {dff.iloc[row].get('name', 'Unknown')}"),
                html.P(f"Age: {dff.iloc[row].get('age_upon_outcome', 'Unknown')}")
            ])
        ])
    ])

###########################
# Run Server
###########################
if __name__ == '__main__':
    logging.info("Starting the dashboard server...")
    app.run_server(mode="external", debug=True, port=8050)


2024-12-16 05:15:10,868 [INFO]: Starting CS-340 Dashboard setup...
2024-12-16 05:15:10,869 [INFO]: Attempting MongoDB connection...
2024-12-16 05:15:10,889 [INFO]: Connected to MongoDB successfully.
2024-12-16 05:15:10,890 [INFO]: Loading initial data from MongoDB...
2024-12-16 05:15:11,019 [INFO]: Initial data loaded successfully.
2024-12-16 05:15:11,022 [INFO]: Logo image loaded successfully.
2024-12-16 05:15:11,033 [INFO]: Initializing Dashboard layout...
2024-12-16 05:15:11,113 [INFO]: Starting the dashboard server...


Dash app running on http://127.0.0.1:8050/
